In [1]:
import torch

In [42]:
# given a tensor of weights (parameters)
weights = torch.tensor([[2.3109,     420.4,     344.003],
                        [0     ,  197.0466,     -233.67],
                        [41.544,     -78  ,       -20.8]], dtype=torch.float32)

# for int8
q_min = -128
q_max = 127

# define r min and r max
r_min = weights.min().item()
r_max = weights.max().item()

# calc s & z
s = (r_max - r_min) / (q_max - q_min)
z = int(round(q_min - r_min/s))

def quantize(tensor):
  result = torch.div(tensor, s) + z
  return result.to(torch.int8)

def dequantize(tensor):
  tensor_as_fp32 = tensor.to(torch.float32)
  return (tensor_as_fp32 - z) * s

# weights -> quantized weights
quantized_weights = quantize(weights).to(torch.int8)

# quantized weights -> weights
dequantized_weights = dequantize(quantized_weights)

# calc the error
quantize_err = abs(weights - dequantized_weights)

print(quantize_err)
print('\n')
err = torch.sum(quantize_err).item()
print(f"Sum of errors = {round(err, 6)}")

tensor([[0.2541, 2.3082, 0.2956],
        [0.0000, 2.1081, 0.2568],
        [2.0607, 1.0506, 0.2802]])


Sum of errors = 8.614197
